In [33]:
from kneed import DataGenerator, KneeLocator
import matplotlib.pyplot as plt
import pandas as pd
import csv
import os
import scipy.io
import json
import numpy as np
from format_data import *
from utils import *
import pickle
from utils_tt import *
import glob
from kneed import DataGenerator, KneeLocator
from quick_extract import *
from get_data import *
from load_rhd import *
import matplotlib.pyplot as plt
from ExtractRecordings.manual.simple_sort import*
import pandas as pd
from PostProcessing.tools.utils import *
import csv
from format_data import *
import pandas as pd
from create_data import *
import os
import glob
import scipy.io
from delta_frequency import *
import math
from utils import *
from format_data import get_sem
from convert_positions_in_tones import *
import re
import json
import pickle
import os
import glob
from tonotopy import *
from utils_tt import *

In [34]:
t_pre = 0.2#0.2
t_post = 0.50#0.300
bin_width = 0.005
# Créer les bins de temps"
psth_bins = np.arange(-t_pre, t_post, bin_width)
#gc = np.arange(0, 32)

In [44]:
path = '/auto/data2/eTheremin/ALTAI/ALTAI_20240724_SESSION_02/'

In [45]:
   # get triggers
triggers_tr, tones_total_tr = get_triggers(path+'headstage_0/', analog_line=0)
triggers_pb, tones_total_pb = get_triggers(path+'headstage_0/', analog_line=1)
        
condition_tr = np.zeros(len(triggers_tr))
condition_pb = np.ones(len(triggers_pb)) 
            
trig_times = np.concatenate((triggers_tr, triggers_pb)) 
tones = np.concatenate((tones_total_tr, tones_total_pb))
condition = np.concatenate((condition_tr, condition_pb))

In [46]:
triggers_mck, mock_tones = [], []
# we have triggers now let's get the tones
json_path = find_json(path)
extracted_data = read_json_file(json_path)
tones_from_files, labels, mock_tones = concatenate_tones_and_labels(extracted_data, path+'headstage_0/tones', False)
#condition, block = convert_condition_block(tones, labels)


4822 4822


In [47]:
print(len(tones))
print(len(trig_times))
print(len(tones_from_files))

9643
9643
9644


In [10]:
print(len(sorted_tones))
print(len(sorted_triggers))

5721
5721


In [12]:
triggers_mck, mock_tones = [], []
# we have triggers now let's get the tones
json_path = find_json(path)
extracted_data = read_json_file(json_path)
tones, labels, mock_tones = concatenate_tones_and_labels(extracted_data, path+'headstage_0/tones', False)
condition, block = convert_condition_block(tones, labels)
save_tt(tones, sorted_triggers, block, condition, triggers_mck, mock_tones, path+'headstage_0')

Tracking
tail length =  5749


In [31]:


# Liste tous les fichiers et dossiers dans le répertoire
fichiers = os.listdir(path+'tones/')

# Filtrer uniquement les fichiers (exclure les sous-dossiers)
n=0
for file in fichiers:
    tones = np.fromfile(path+ 'tones/'+file, dtype=np.double)
    print(tones)
    n+=len(tones)



[3109. 2684. 2317. ...  715.  828.  715.]


In [32]:
n

5749

In [102]:
tt_path = path+'headstage_0/tt.pkl'
with open(tt_path, 'rb') as file:
    tt = pickle.load(file)
print(len(tt['triggers']))
print(len(tt['tones']))

5721
5749


In [22]:
triggers_tr, tones_total_tr = get_triggers(path+'headstage_0/', analog_line=0)
triggers_pb, tones_total_pb = get_triggers(path+'headstage_0/', analog_line=1)
triggers_mock, tones_total_mock = get_triggers(path+'headstage_0/', analog_line=2)

In [27]:
len(tones_total_pb)

0

In [34]:
condition_tr = np.zeros(len(triggers_tr))
condition_pb = np.ones(len(triggers_pb)) 
            
trig_times = np.concatenate((triggers_tr, triggers_pb)) 
tones = np.concatenate((tones_total_tr, tones_total_pb))
condition = np.concatenate((condition_tr, condition_pb))
            

print(len(tones))
print(len(trig_times))
    
#sorted_indices = np.argsort(trig_times[:len(tones)])
sorted_indices = np.argsort(trig_times)
sorted_indices = sorted_indices[:-1]
sorted_triggers = trig_times[sorted_indices]
sorted_tones = tones[sorted_indices]
sorted_condition = condition[sorted_indices]

0
14547


IndexError: index 0 is out of bounds for axis 0 with size 0

In [32]:
condition

array([0., 0., 0., ..., 1., 1., 1.])

In [18]:
tt = {
    'tones': sorted_tones,
    'triggers': sorted_triggers, 
    'condition' : sorted_condition,
    'block': np.full(len(sorted_condition), "Block_00")
    }
     
    # save tt.pkl
with open(path+'/tt.pkl', 'wb') as file:
    pickle.dump(tt, file)

In [14]:
tt_path = path+'headstage_1/tt.pkl'
with open(tt_path, 'rb') as file:
    tt = pickle.load(file)
print(len(tt['triggers']))
print(len(tt['tones']))

FileNotFoundError: [Errno 2] No such file or directory: '/auto/data2/eTheremin/BURRATA/BURRATA/BURRATA_20240426_SESSION_00/headstage_1/tt.pkl'

In [168]:
spk_times = np.load(path+'headstage_1/spike_times.npy')

In [ ]:
print(spk_times)

In [153]:
def quick_extract(path, mode="relative", threshold=-3.7):
    root_dir = os.path.split(path)[0]
    data = ss.load_spike_data(path)
    channels = np.arange(data.shape[0])
    spike_times = np.empty(0, dtype=np.uint64)
    spike_clusters = np.empty(0, dtype=np.int32)
    assert mode in ("relative", "absolute"), "Mode is relative (from RMS calculation) or absolute (threshold in µV)."
    to_float = False
    if data.dtype == np.uint16:
        to_float = True
    if mode == "absolute":
        threshold = -60
    else:
        threshold = threshold
    for i, channel in tqdm(enumerate(channels)):
        if to_float:
            chan = np.multiply(0.195, (data[channel].astype(np.int32) - 32768))
            spk, _ = ss.thresholder(chan, mode, threshold=threshold)
        else:
            spk, _ = ss.thresholder(data[channel], mode, threshold=threshold)
        cluster = np.full_like(spk, i)
        spike_times = np.hstack((spike_times, spk))
        spike_clusters = np.hstack((spike_clusters, cluster))
    idx = np.argsort(spike_times)
    spike_times = spike_times[idx]
    spike_clusters = spike_clusters[idx]
    np.save(os.path.join(root_dir, "spike_times.npy"), spike_times)
    np.save(os.path.join(root_dir, "spike_clusters.npy"), spike_clusters)
    pass

In [ ]:
quick_extract(path+'headstage_1/neural_data.npy', mode="relative", threshold=-3.7)

In [ ]:
spk_times = np.load(path+'/spike_times.npy', allow_pickle=True )
spk_clusters = np.load(path+'/spike_clusters.npy',allow_pickle=True)

clusters = {}
for value, cluster in zip(spk_times, spk_clusters):
    if cluster not in clusters:
        clusters[cluster] = []
    clusters[cluster].append(value)
for cluster, values in clusters.items():
    print(f"Cluster {cluster}: {len(values)}")

In [ ]:
import os

# Définir le chemin vers le répertoire contenant les fichiers
repertoire = path+'headstage_1/tones/'

# Définir le modèle de nom de fichier, avec le numéro variant de 01 à 06
modele = "tones_playback_{}_BURRATA_SESSION_00_20240430.bin"

n_tones = 0
# Parcourir les fichiers du répertoire
for numero in range(1, 7):  # Les numéros de 01 à 06
    nom_fichier = modele.format(str(numero).zfill(2))  # zfill(2) pour ajouter un zéro initial
    chemin_fichier = os.path.join(repertoire, nom_fichier)
    
    # Vérifier si le fichier existe
    if os.path.exists(chemin_fichier):
        print(f"Ouverture du fichier : {chemin_fichier}")
        contenu = np.fromfile(chemin_fichier, dtype=np.double)
        n_tones+=len(contenu)
            # Faire quelque chose avec le contenu du fichier



In [ ]:
tr = np.load('/auto/data2/eTheremin/BURRATA/BURRATA/BURRATA_20240522_SESSION_00/headstage_1/trig_analog_chan_MAIN.npy')
pb = np.load('/auto/data2/eTheremin/BURRATA/BURRATA/BURRATA_20240522_SESSION_00/headstage_1/trig_analog_chan_PLAYBACK.npy')

In [35]:
#convertir les tt.npz en tt.pkl avec le bloc et la condtiion et tout
tt = np.load('/auto/data2/eTheremin/FRINAULT/FRINAULT_20230218/FRINAULT_20230218_SESSION_00/tt.npz')

In [36]:
tt

NpzFile '/auto/data2/eTheremin/FRINAULT/FRINAULT_20230218/FRINAULT_20230218_SESSION_00/tt.npz' with keys: order, n_iter, recording_length, keys, numbers...

In [37]:


# List the titles/keys in the .npz file
keys = tt.keys()
print("Titles in the .npz file:", keys)

# Iterate over the keys and print the corresponding values
for key in keys:
    print(f"Values under '{key}':\n", tt[key])


Titles in the .npz file: KeysView(NpzFile '/auto/data2/eTheremin/FRINAULT/FRINAULT_20230218/FRINAULT_20230218_SESSION_00/tt.npz' with keys: order, n_iter, recording_length, keys, numbers...)
Values under 'order':
 [ 3  6  9 12 15 18 21 24 27 30  0 31  1  4  7 10 13 16 19 22 25 28  2  5
  8 11 14 17 20 23 26 29]
Values under 'n_iter':
 [10]
Values under 'recording_length':
 [220487424]
Values under 'keys':
 [b'pb_0' b'pb_1' b'pb_2' b'pb_3' b'pb_4' b'pb_5' b'pb_6' b'pb_7' b'pb_8'
 b'pb_9' b'wp_0' b'wd_0' b'tr_0' b'tr_1' b'tr_2' b'tr_3' b'tr_4' b'tr_5'
 b'tr_6' b'tr_7' b'tr_8' b'tr_9' b'mk_0' b'mk_1' b'mk_2' b'mk_3' b'mk_4'
 b'mk_5' b'mk_6' b'mk_7' b'mk_8' b'mk_9']
Values under 'numbers':
 [0 1 2 3 4 5 6 7 8 9 0 0 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]
Values under 'pb_0':
 [[8.2800000e+02 9.5900000e+02 8.2800000e+02 9.5900000e+02 8.2800000e+02
  9.5900000e+02 8.2800000e+02 9.5900000e+02 8.2800000e+02 9.5900000e+02
  8.2800000e+02 9.5900000e+02 8.2800000e+02 9.5900000e+02 8.2800000e+02


In [40]:
tt['pb_0'][0]

array([ 828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,
        959.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,
        828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,
        959.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,
        828.,  959.,  828.,  715.,  617.,  715.,  617.,  715.,  617.,
        715.,  617.,  715.,  828.,  959.,  828.,  959.,  828.,  715.,
        617.,  715.,  617.,  715.,  617.,  715.,  617.,  715.,  617.,
        533.,  617.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,
        617.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,  617.,
        533.,  617.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,
        617.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,  617.,
        533.,  617.,  533.,  617.,  715.,  617.,  533.,  617.,  533.,
        617.,  533.,  617.,  533.,  617.,  715.,  617.,  715.,  617.,
        715.,  617.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,
        617.,  533.,

In [72]:
path = '/auto/data2/eTheremin/FRINAULT/FRINAULT_20230218/FRINAULT_20230218_SESSION_00/'
tt = np.load(path+'tt.npz')

keys = tt.keys()

tr_triggers, tr_tones = [], []
pb_triggers, pb_tones = [], []
condition, block = [], []

for key in keys:
    key_str = str(key) 
    # S'assurer que le titre a au moins 3 caractères
    if len(key_str) >= 3:
        c = key_str[:2] #extraure la condition
        b = key_str[-1] # extraire le block
    if c=='pb':
        pb_triggers.append(tt[key][1])
        pb_tones.append(tt[key][0])
        condition.append(np.ones_like(tt[key][0]))
        block.append(np.full(tt[key][0].shape, f'Block_00{int(b)}', dtype=object))
    elif c=='tr':
        tr_triggers.append(tt[key][1])
        tr_tones.append(tt[key][0])
        condition.append(np.zeros_like(tt[key][0]))
        block.append(np.full(tt[key][0].shape, f'Block_00{int(b)}', dtype=object))

trig_times = np.concatenate((np.hstack(tr_triggers), np.hstack(pb_triggers))) 
tones = np.concatenate((np.hstack(tr_tones), np.hstack(pb_tones)))
condition = np.hstack(condition)
block = np.hstack(block)

sorted_indices = np.argsort(trig_times)
print(sorted_indices)
sorted_indices = sorted_indices[:-1]
sorted_triggers = trig_times[sorted_indices]
sorted_tones = tones[sorted_indices]
sorted_condition = np.array(condition)[sorted_indices]
sorted_block = np.array(block)[sorted_indices]

tt = {
    'tones': sorted_tones,
    'triggers': sorted_triggers, 
    'condition' : sorted_condition,
    'block': sorted_block
    }
     
    # save tt.pkl
with open(path+'headstage_0/tt.pkl', 'wb') as file:
    pickle.dump(tt, file)

    

[   0    1    2 ... 9751 9752 9753]


In [73]:
#tt_path = path+'/tt.pkl'
tt_path = '/auto/data2/eTheremin/FRINAULT/FRINAULT_20230218/FRINAULT_20230218_SESSION_00/headstage_0/tt.pkl'
with open(tt_path, 'rb') as file:
    tt = pickle.load(file)
print(tt)
print(tt['block'])

{'tones': array([959., 828., 959., ..., 533., 617., 533.]), 'triggers': array([2.06307110e+07, 2.06327110e+07, 2.06346910e+07, ...,
       2.00165109e+08, 2.00171128e+08, 2.00186068e+08]), 'condition': array([1., 1., 1., ..., 0., 0., 0.]), 'block': array(['Block_000', 'Block_000', 'Block_000', ..., 'Block_009',
       'Block_009', 'Block_009'], dtype=object)}
['Block_000' 'Block_000' 'Block_000' ... 'Block_009' 'Block_009'
 'Block_009']


In [13]:
neural = np.load(path+'filtered_neural_data.npy', allow_pickle=True)
#neural = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/neural_data.npy', allow_pickle=True)

In [14]:
data = np.load(path+f'/data_{bin_width}.npy', allow_pickle=True)
features = np.load(path+f'/features_{bin_width}.npy', allow_pickle=True)
gc = np.load(path+'/good_clusters.npy', allow_pickle=True)

In [ ]:
gc

In [ ]:
plt.plot(data[1])

In [ ]:
t_chelou = int(395000*0.005)*30000
print(t_chelou)

In [21]:
neural = np.transpose(neural)

In [ ]:
plt.plot(neural[1][t_chelou-10000:t_chelou+10000])

In [ ]:
plt.plot(neural[1][t_chelou-1000:t_chelou+1000])

In [ ]:
n = int(len(neural[0])/2)
print(n)

In [67]:
spk_times = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_times.npy')
spk_clus = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_clusters.npy')
spk_26 = [spk_times[i] for i in range(len(spk_clus)) if spk_clus[i] == 26]

In [ ]:
plt.figure(figsize=(15, 8)) 
plt.plot(neural[1][:n], c='red')
#plt.scatter(spk_26[:15000], np.full_like(spk_26[:15000], 500))


In [ ]:
plt.figure(figsize=(15, 8)) 
plt.plot(neural[1][n:], c='red')
#plt.scatter(spk_26[:15000], np.full_like(spk_26[:15000], 500))


In [37]:
indices = np.where((neural[1] > 2000) | (neural[1]  < -2000))

In [ ]:
indices

In [31]:

spk_times = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_times.npy')
spk_clus = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_clusters.npy')

spike_times = [spk_times[i] for i in range(len(spk_times)) if spk_clus[i] == 26]
tt_path = path+'/tt.pkl'
with open(tt_path, 'rb') as file:
    tt = pickle.load(file)
stimulus_times = [elt for elt in tt['triggers']]
# Paramètres pour le PSTH
bin_size = 0.01  # Taille des bins (ex: 10 ms)
pre_stimulus_time = -0.5  # Temps avant le stimulus (ex: 500 ms avant)
post_stimulus_time = 0.5  # Temps après le stimulus (ex: 500 ms après)
num_bins = int((post_stimulus_time - pre_stimulus_time) / bin_size)  # Nombre de bins

# Liste pour stocker les spikes alignés par rapport aux stimuli
all_aligned_spikes = []

# Pour chaque stimulus, aligner les spikes par rapport au stimulus
for stim_time in stimulus_times:
    # Calculer le temps des spikes par rapport à chaque stimulus
    aligned_spikes = spike_times - stim_time
    # Garder uniquement les spikes dans la fenêtre [-pre_stimulus_time, post_stimulus_time]
    spikes_in_window = aligned_spikes[(aligned_spikes >= pre_stimulus_time) & (aligned_spikes <= post_stimulus_time)]
    # Ajouter ces spikes à la liste
    all_aligned_spikes.append(spikes_in_window)

# Combiner tous les spikes alignés dans une seule liste
all_aligned_spikes = np.concatenate(all_aligned_spikes)

In [16]:
def quick_extract(path, mode="relative", threshold=-3.7):
    root_dir = os.path.split(path)[0]
    data = ss.load_spike_data(path)
    data = np.transpose(data)
    channels = np.arange(data.shape[0])
    spike_times = np.empty(0, dtype=np.uint64)
    spike_clusters = np.empty(0, dtype=np.int32)
    assert mode in ("relative", "absolute"), "Mode is relative (from RMS calculation) or absolute (threshold in µV)."
    to_float = False
    if data.dtype == np.uint16:
        to_float = True
    if mode == "absolute":
        threshold = -60
    else:
        threshold = threshold
    for i, channel in tqdm(enumerate(channels)):
        if to_float:
            chan = np.multiply(0.195, (data[channel].astype(np.int32) - 32768))
            spk, _ = ss.thresholder(chan, mode, threshold=threshold)
        else:
            spk, _ = ss.thresholder(data[channel], mode, threshold=threshold)
        cluster = np.full_like(spk, i)
        spike_times = np.hstack((spike_times, spk))
        spike_clusters = np.hstack((spike_clusters, cluster))
    idx = np.argsort(spike_times)
    spike_times = spike_times[idx]
    spike_clusters = spike_clusters[idx]
    np.save(os.path.join(root_dir, "spike_times.npy"), spike_times)
    np.save(os.path.join(root_dir, "spike_clusters.npy"), spike_clusters)
    pass

In [ ]:
quick_extract('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/filtered_neural_data.npy', threshold=-3)

In [11]:
spk_times_fp = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_times_fp.npy')
spk_clus_fp = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_clusters_fp.npy')

In [27]:
spk_times = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_times.npy')
spk_clus = np.load('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/spike_clusters.npy')

In [28]:

# Filtrer les éléments dont l'indice est 3
spk_26_fp = [spk_times_fp[i] for i in range(len(spk_clus_fp)) if spk_clus_fp[i] == 26]

spk_26 = [spk_times[i] for i in range(len(spk_clus)) if spk_clus[i] == 26]


In [ ]:
print(len(spk_26_fp), len(spk_26))

In [ ]:
quick_extract('/auto/data2/eTheremin/ALTAI/ALTAI_20240910_SESSION_00/headstage_0/filtered_neural_data.npy')

In [3]:
triggers_tr, tones_total_tr = get_triggers(path, analog_line=0)
triggers_pb, tones_total_pb = get_triggers(path, analog_line=1)

In [ ]:
plt.plot(triggers_tr, c = 'red')
plt.plot(triggers_pb, c = 'purple')

In [5]:
gc = np.load(path+'headstage_0/good_clusters.npy')

In [ ]:
gc

In [4]:
dig = np.load('/auto/data2/eTheremin/MUROLS/MUROLS_20230220/MUROLS_20230220_SESSION_00/dig_in.npy', allow_pickle=True)
an = np.load('/auto/data2/eTheremin/MUROLS/MUROLS_20230220/MUROLS_20230220_SESSION_00/analog_in.npy', allow_pickle=True)

In [5]:
an_triggers = np.load(os.path.join(path, "analog_in.npy"))
an_times = ut.extract_analog_triggers_compat(an_triggers[0])

In [ ]:
an1_times = ut.extract_analog_triggers_compat(an_triggers[1])
len(an_times)

In [ ]:
len(an1_times)

In [ ]:
def get_tracking_files(folder):
    # Trouver tous les fichiers qui comportent "tracking" dans leur nom
    all_files = glob.glob(os.path.join(folder, "tracking_0*.bin"))
    
    # Afficher les fichiers trouvés
    print("Files matching the pattern:")
    for file in all_files:
        print(file)
    
    return all_files

# Exemple d'utilisation
folder = "/auto/data2/eTheremin/MUROLS/MUROLS_20230220/MUROLS_20230220_SESSION_00"
tracking_files = get_tracking_files(folder)

In [ ]:
plt.plot(an1_times)
plt.plot(an_times)

In [12]:

def get_tracking_tones_verif(folder):
    all_files = glob.glob(os.path.join(folder, "tracking_0*.bin"))
    print(all_files)
    print(len(all_files))
    for file in all_files:
        print(file)
        print(extract_number_from_filename(file, 'tracking_'))
    all_files.sort(key=lambda x: extract_number_from_filename(x, 'tracking_'))
    # Print all matching files
    print("Files matching the pattern:")
    for file in all_files:
        print(file)
    all_tones, all_blocs = [], []
    for file in all_files:
        
        # Load the binary file into a NumPy array
        tones = np.fromfile(file, dtype=np.double)
        
        # Append the tones data to the list
        all_tones.append(tones)
        blocs = np.full(len(tones),extract_number_from_filename(file, 'tracking_'))
        all_blocs.append(blocs)
    return all_tones, all_blocs

def get_mock_tones_verif(folder):
    all_files = glob.glob(os.path.join(folder, "*mock_*.bin"))
    print(all_files)
    all_files.sort(key=lambda x: extract_number_from_filename(x, 'mock_'))
    # Print all matching files
    print("Files matching the pattern:")
    for file in all_files:
        print(file)
    all_tones, all_blocs = [], []
    for file in all_files:
        
        # Load the binary file into a NumPy array
        tones = np.fromfile(file, dtype=np.double)
        
        # Append the tones data to the list
        all_tones.append(tones)
        blocs = np.full(len(tones),extract_number_from_filename(file, 'mock_'))
        all_blocs.append(blocs)
    return all_tones, all_blocs

def get_playback_tones_verif(folder):
    # Find all files matching the pattern
    all_files = glob.glob(os.path.join(folder, "playback_*.bin"))

    # Print all matching files
    print("Files matching the pattern:")
    for file in all_files:
        print(file)

    # Sort files by the number extracted from their filename
    all_files.sort(key=lambda x: extract_number_from_filename(x, 'playback_'))

    all_tones, all_blocs = [], []
    for file in all_files:
        # Load the binary file into a NumPy array
        tones = np.fromfile(file, dtype=np.double)
        
        # Append the tones data to the list
        all_tones.append(tones)
        
        # Create an array filled with the file number
        blocs = np.full(len(tones), extract_number_from_filename(file, 'playback_'))
        all_blocs.append(blocs)

    return all_tones, all_blocs


In [ ]:
playback, block = get_playback_tones_verif(path)

In [ ]:
mock, block = get_mock_tones_verif(path)

In [ ]:
len(np.hstack(mock))

In [ ]:
3741+3742

In [ ]:
extract_number_from_filename('/auto/data2/eTheremin/MUROLS/MUROLS_20230220/MUROLS_20230220_SESSION_00/tracking_05_MUROLS_SESSION_00_20230220.bin', 'tracking_')

In [ ]:
import os

# Spécifie le chemin du dossier contenant les fichiers
#folder_path = path+'headstage_0/tones/'

# Parcours tous les fichiers du dossier
for filename in os.listdir(folder_path):
    # Construire le chemin complet des fichiers
    old_file_path = os.path.join(folder_path, filename)

    # Vérifie que c'est bien un fichier (et non un sous-dossier)
    if os.path.isfile(old_file_path):
        # Nouveau nom de fichier avec 'tones_' ajouté au début
        new_filename = "tones_" + filename
        new_file_path = os.path.join(folder_path, new_filename)

        # Renomme le fichier
        os.rename(old_file_path, new_file_path)
        print(f"Renommé : {filename} -> {new_filename}")

In [77]:
tt_path = path+'headstage_0/tt.pkl'
with open(tt_path, 'rb') as file:
    tt = pickle.load(file)

In [78]:
triggers = [elt for elt in tt['triggers'] ]
tones = [elt for elt in tt['tones'] ]
condition = [elt for elt in tt['condition']]
block = [elt for elt in tt['block']]
block = [int(elt[-1]) for elt in block]

In [ ]:
block

In [66]:
import numpy as np

def extract_triggers_and_tones(triggers, tones, condition, block, condition_value, block_value):
    # Vérifie que les tableaux ont la même longueur
    
    # Trouver les indices où condition == condition_value et block == block_value
    indices = np.where((condition == condition_value) & (block == block_value))[0]
    # Extraire les valeurs correspondantes dans triggers et tones
    triggers_filtered = triggers[indices]
    tones_filtered = tones[indices]
    
    return triggers_filtered, tones_filtered


In [ ]:
for b in range(1,6):   
    print(b)
    tr_tr, tr_tones = extract_triggers_and_tones(np.array(triggers), np.array(tones), np.array(condition), np.array(block), 0, b)
    tr_pb, tones_pb = extract_triggers_and_tones(np.array(triggers), np.array(tones), np.array(condition), np.array(block), 1, b)
    plt.plot(tr_tr)
    plt.plot(tr_pb-(tr_tr[-1]-tr_tr[0]))
    #plt.plot(tones_pb)
    #plt.plot(tr_tones)
    plt.show()

In [ ]:
plt.plot(tr_tr)
plt.plot(tr_pb-(tr_tr[-1]-tr_tr[0]))

In [ ]:
print(tr_tr)
print(tr_pb-(tr_tr[-1]-tr_tr[0]))

In [84]:
triggers_tr, tones_total_tr = get_triggers(path+'headstage_0/', analog_line=0)
triggers_pb, tones_total_pb = get_triggers(path+'headstage_0/', analog_line=1)

In [ ]:
len(triggers_tr)-len(np.hstack(tones))

In [ ]:
len(tones_total_pb)

In [94]:
def get_mock_tones_verif(folder):
    all_files = glob.glob(os.path.join(folder+'/tones/mock/', "*mock*.bin"))
    all_files.sort(key=lambda x: extract_number_from_filename(x, 'mock_'))
    # Print all matching files
    print("Files matching the pattern:")
    for file in all_files:
        print(file)
    all_tones, all_blocs = [], []
    for file in all_files:
        
        # Load the binary file into a NumPy array
        tones = np.fromfile(file, dtype=np.double)
        
        # Append the tones data to the list
        all_tones.append(tones)
        blocs = np.full(len(tones),extract_number_from_filename(file, 'mock_'))
        all_blocs.append(blocs)
    return all_tones, all_blocs

In [ ]:
tones, block = get_mock_tones_verif(path+'headstage_0')

In [ ]:
len(np.hstack(tones))

In [14]:
condition_tr = np.zeros(len(triggers_tr))
condition_pb = np.ones(len(triggers_pb))

In [15]:
trig_times = np.concatenate((triggers_tr, triggers_mc))
tones = np.concatenate((tones_total_tr, tones_total_mc))
condition = np.concatenate((condition_tr, condition_mc)) 

In [19]:
sorted_indices = np.argsort(trig_times[:len(tones)])

In [ ]:
len(sorted_indices)

In [21]:
sorted_indices = sorted_indices[:-1]
sorted_triggers = trig_times[sorted_indices]
sorted_tones = tones[sorted_indices]
sorted_condition = condition[sorted_indices]
block = creer_tableau_blocs(sorted_condition)


In [ ]:
plt.plot(sorted_condition)

In [63]:
def creer_tableau_blocs(A):
    # Initialiser un tableau pour stocker les numéros de blocs
    blocs = np.zeros_like(A, dtype=int)  # Un tableau de la même taille que A, rempli de zéros
    bloc_courant = 0  # Compteur du bloc
    
    # Parcourir le tableau A
    for i in range(1, len(A)):
        # Si on passe de 0 à une valeur non-nulle, on démarre un nouveau bloc
        if A[i-1] != 0 and A[i] == 0 and A[i+1] == 0:
            print(A[i-1], A[i], A[i+1])
            bloc_courant += 1  # Nouveau bloc trouvé

        # Affecter le numéro du bloc courant aux éléments non-nuls
        if A[i] != 0:
            blocs[i] = bloc_courant
        else:
            blocs[i] = bloc_courant  # Les zéros appartiennent à aucun bloc (ou bloc zéro)
    
    return blocs

In [ ]:
block = creer_tableau_blocs(sorted_condition)

In [ ]:
plt.plot(block)
plt.plot(sorted_condition)

In [27]:
json_path = find_json(path)
extracted_data = read_json_file(json_path)
tones, labels, mock_tones = concatenate_tones_and_labels(extracted_data, path+'headstage_0/tones')
condition, block = convert_condition_block(tones, labels)

In [ ]:
extracted_data